# 事前準備
安裝 `bs4` [pypi.org](https://pypi.org/project/beautifulsoup4/)


安裝 `selenium` 及對應的瀏覽器驅動，詳細的安裝頁面可以在 [pypi.org](https://pypi.org/project/selenium/) 中找到。

在此程式碼中皆以 [chrome](https://chromedriver.chromium.org/downloads) 作為操作瀏覽器。

進入到下載頁面後 點選最新版本的驅動下載連結，根據對應的作業系統點選下載點
`ChromeDriver 114.0.5735.90 > chromedriver_win32.zip`

將內容解壓縮到跟程式碼相同的目錄下底便完成了事前準備。

In [578]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select # dropdown element

[國立成功大學課程資訊及選課系統](https://course.ncku.edu.tw/) 左上 「課程資訊$\rightarrow$歷年課程查詢」

打開新頁面後點選 [歷年課程進階查詢](https://course-query.acad.ncku.edu.tw/index.php?c=qry11215&m=en_query) 複製其網址串，如：https://course-query.acad.ncku.edu.tw/index.php?c=qry11215&m=en_query

In [582]:
# 要查詢的課程名稱
course_name = '人工智慧'

# 查詢年份
year_start = 111
year_end = 112

In [577]:
driver = webdriver.Chrome()

url = "https://course-query.acad.ncku.edu.tw/index.php?c=qry11215&m=en_query"

driver.get(url)

req = requests.get(url)

In [583]:
# html 中的拉條功能(dropdown) 選擇 "年份" 要使用Select來控制 
year_start_selector = Select(driver.find_element_by_xpath('//*[@id="sel_syear_b"]'))
year_end_selector = Select(driver.find_element_by_xpath('//*[@id="sel_syear_e"]'))

year_start_selector.select_by_value(f'{year_start}')
year_end_selector.select_by_value(f'{year_end}')

# 要查詢的課程名稱
course_name = '人工智慧'

course_textBox = driver.find_element_by_xpath('//*[@id="cosname"]')
course_textBox.send_keys(f'{course_name}')

In [584]:
button = driver.find_element_by_xpath("/html/body/div[4]/div/div[2]/button")
button.click()

在 `button.click()` 執行後，頁面會根據輸入的 `course_name` 、 `year_start` 跟 `year_end` 轉跳頁面。

轉跳完頁面後，當前的 url 會跟著改動。要擷取查詢後頁面的結果，需要對目前 url 發出 `get` 的請求。

這通常需要幾秒鐘的時間等待完成(根據本身網速以及學校網路)，否則直接執行 `requests.get()` 會導致請求的頁面資訊不完整。

In [585]:
current_url = driver.current_url

In [586]:
# 需要等待幾秒等網頁把內容載入好 才能得到完整資訊
searched_web = requests.get(current_url)

# 解析查詢頁面
轉跳完查詢頁面後，我們只關心課程內容。可以在Chrome頁面案右鍵選擇"檢查"後，可以看到整個html語法。
    
整個html的是樹狀形式的，查詢的內容以 CSS selector 表示為 `#main_content > div.visible-xs` ，其子節點為 &lt;table class="table table-bordered"&gt;

課程內容以 6 個 <tr> 為單位，依序為
    
    1. 系號-序號 課程碼-分班碼 屬性碼
    
    2. 科目名稱 備註 限選條件
    
    3. 時間/教室
    
    4. 已選課人數/餘額
    
    5. &lt;tr class="tr-info tr-info-0"&gt;
    
    6. &lt;tr class="tr-info tr-info-0"&gt;




    
實際上需要的只有 1. 2. 4. 6. 的內容

In [581]:
# BeautifulSoup 解析 html 
soup = BeautifulSoup(searched_web.text, "html.parser")

# 由 CSS selectors 定位課程資訊的位置
table = soup.select_one('#main_content > div.visible-xs > table > tbody')

# <tr> 代表 table row 
# table row 中包含 <th> <td> 個別對應標題跟資料
tr_nodes = table.find_all('tr', recursive=False)

def extract_course_codes(tr_node):
    # 開課代碼
    codes = tr_node.find('td').text.split()
    
    if not codes:
        raise ValueError("Course codes are empty.")

    return codes

def extract_course_name_and_programs(tr_node):
    # 課程名稱 學程
    course_name = tr_node.find('span', class_='course_name').get_text()
    program = tr_node.find('div', class_='label label-success ips')
    if program:
        program_name = program.get_text()
        return [course_name, program_name]
    else:
        return [course_name, '']

def extract_number_of_people(tr_node):
    # 修課人數
    # 忽略 <span> 標籤內容
    if tr_node.find('span'):
        tr_node.find('span').clear()
    # 儲存成 .csv 等檔案 由於會把格式自動轉換成日期格式
    # 所以將字串處理成 選課人數,餘額
    number_of_people = tr_node.contents[1].text.split('/')
    return number_of_people

def extract_tr_info(tr_node):
    # 其他 開課系所 課程大綱 教師名稱
    td_tags = tr_node.find_all('tr')
    target_student = td_tags[1].contents[1].get_text(strip=True)
    if '碩' in target_student:
#         print('碩班課程 剔除')
        return False
    
    department_name = td_tags[0].contents[1].text
    outline_link = td_tags[4].contents[1].a.get('href')
#     outline_link = td_tags[4].contents[1].find('a', href=True)
#     print(outline_link)
    teacher_name = td_tags[5].contents[1].find_all(text=True)[0].replace('*', '')

    return [department_name, outline_link, teacher_name]


course_info = []

# 以 6 個 <tr> 標籤為單位
for i in range(0, len(tr_nodes), 6):
    others_info = extract_tr_info(tr_nodes[i+5])
    if others_info == False:
        continue
    else:
        course_codes = extract_course_codes(tr_nodes[i])
        course_name_and_programes = extract_course_name_and_programs(tr_nodes[i+1])
        number_of_people = extract_number_of_people(tr_nodes[i+3])
        
        course_info.append(course_codes + course_name_and_programes + number_of_people + others_info)


In [573]:
import pandas as pd
index_name = ['系號-序號', '課程碼-分班碼', '屬性碼', '科目名稱', '學程', '已選課人數', '餘額', '開課系所', '課程大綱', '教師名稱']
course_df = pd.DataFrame(course_info, columns=index_name)
course_df

,系號-序號,課程碼-分班碼,屬性碼,科目名稱,學程,已選課人數,餘額,開課系所,課程大綱,教師名稱
0,D0-102,D010500,[CSS5005],人工智慧在社會科學中的應用,社會資料科學學分學程,18,7,社會科學院 CSS,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,胡中凡
1,F7-154,F742900,[CSIE5010],人工智慧導論,人工智慧學分學程,85,45,資訊系 CSIE,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,朱威達
2,F7-160,F743200,[CSIE5013],人工智慧於醫療應用與服務,人工智慧學分學程,102,35,資訊系 CSIE,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,高宏宇
3,H3-165,H335800,[IIM4439],人工智慧導論,FinTech金融科技學分學程,22,8,工資系 IIM,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,李昇暾
4,I5-066,I564700,[MED2308],人工智慧與醫學,醫療器材與系統學程,25,5,醫學系 MED,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,蔡依珊
5,J0-105,J030700,[SOC2006],人工智慧運算架構與系統,智慧運算學分學程,6,額滿,敏求學院課程 SOC,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,林偉棻
6,J0-106,J030100,[SOC1004],人工智慧法律與政策,智慧運算學分學程，社會資料科學學分學程,41,8,敏求學院課程 SOC,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,李韶曼
7,J0-107,M060100,[SOC5004],人工智慧與治理,智慧運算學分學程,20,10,敏求學院課程 SOC,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,李韶曼
8,E1-139,E134400,[ME3129],人工智慧深度學習概論與應用,,59,1,機械系 ME,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,林啟倫
9,E2-094,E222300,[EE2550],人工智慧導論及實作,跨域永續綠能學分學程,49,21,電機系 EE,https://class-qry.acad.ncku.edu.tw/syllabus/sy...,詹寶珠


In [574]:
# 儲存 course_df
# 使用 to_csv 由於中文字編碼的關係 有可能會有亂碼產生
# 使用 pandas 中的 to_excel 需要額外安裝 openxel 套件
# 也可以改用 to_csv
course_df.to_csv('test.csv',encoding="utf_8_sig", index=False)